In [ ]:
import re
import pandas as pd
import csv
import os
import numpy as np
import time

from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Code")
import cleaning_tweets as ct
os.chdir("/content/drive/MyDrive/Data")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def clean_csv(df):
    # Hashtags
    ## clean hashtags
    df['hashtags'] = df['hashtags'].apply(ct.clean_hashtags)
    
    # Geography
    ## remove empty geography
    df.dropna(subset=['geography'], axis=0, how='any', inplace=True)

    ## divide the geography into 'country_code' ,'state', 'county' and 'city'
    gg_cols = ['state', 'county', 'city']
    for gg_value in gg_cols:
        df.loc[:, gg_value] = df['geography'].apply(lambda r: ct.clean_geography(r, gg_value))
    
    ## drop nan value and combine
    df.dropna(subset=['county', 'city'], axis=0, how='all', inplace=True)
    df.loc[:, 'county and city'] = df.apply(lambda r: r['county'] if pd.isna(r['county'])==False else r['city'], axis=1)
    df.reset_index(inplace=True, drop=True)

    # Full_text
    ## clean tweets
    df.loc[:, 'clean_text'] = df['full_text'].apply(ct.clean_tweets)
    
    df.drop(['full_text', 'geography', 'county', 'city'], axis=1, inplace=True)
    df.drop_duplicates(subset=['id'], keep='first', inplace=True)
    
    df1 = df.iloc[:, [0, 1, 7, 2, 5, 6, 3, 4]].copy()
    df1['id'] = df1['id'].apply(str)
    df1.dropna(subset=['clean_text'], axis=0, how='any', inplace=True)
    
    return df1

In [ ]:
df_chunk = pd.read_csv('all_gb.csv', header = None, dtype = str, \
                                            chunksize=300000, encoding='utf-8-sig', engine='python')
count = 0
for chunk in df_chunk:
    start_time = time.time()
    chunk.columns = ['date', 'id', 'full_text', 'hashtags', \
                                    'retweet_count', 'favorite_count', 'geography']
    chunk_new = clean_csv(chunk)
    chunk_new.to_csv('cleaned_data.csv', mode='a', header=None, index=False, encoding='utf-8-sig')
    print(f"Done: {count} \n time: {(time.time() - start_time)//60} minutes")
    count += 1

Generate unique text with text_id

In [ ]:
df = pd.read_csv('cleaned_data.csv', header = None, dtype = str, encoding = 'utf-8-sig')
df.columns = ['date', 'id', 'clean_text', 'hashtags', \
                        'state', 'county and city',\
                        'retweet_count', 'favorite_count']
print(f"df has {df.shape[0]} rows")

text_unique = list(np.unique(df['clean_text'].astype(str)))

df_text = pd.DataFrame(text_unique, columns=['text'])
df_text = pd.concat([df_text, pd.DataFrame(list(range(len(text_unique))), columns=['numid'])], axis=1)
df_text.loc[:, "text_id"] = df_text["numid"].astype(str).apply(lambda r: 'text_' + r)
df_text.iloc[:, [0, 2]].to_csv('textid.csv', mode='a', header=None, index=False, encoding = 'utf-8-sig')
print(f"textid.csv has {df_text.shape[0]} rows")